<a href="https://colab.research.google.com/github/A-NGJ/AudioExplorers2023/blob/main/AudioExplorers2023_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

def resize_data(data):
    resized_data = np.zeros((data.shape[0], 32, 96))
    max_pixel_value = data.max()
    min_pixel_value = data.min()
    for i, img in enumerate(data):
        resized_img = cv2.resize(img, (96, 32))
        resized_img = (resized_img - min_pixel_value) / (max_pixel_value - min_pixel_value)  # Normalize pixel values to [0, 1]
        resized_data[i] = resized_img
    return resized_data

# # Load the data
# data = np.load("training.npy")
# labels = np.load("training_labels.npy")

# # data = resize_data(data)

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
# Load data
X = np.load('training.npy')
y = np.load('training_labels.npy')

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_test, y_val = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)


X_train = np.load("X_train_aug.npy")
y_train = np.load("y_train_aug.npy")
# Preprocess the data
# X_train = X_train.astype("float32")
# X_val = X_val.astype("float32") 

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

# Define the CNN architecture
model = tf.keras.Sequential([
    # tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 96, 1)),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dense(5, activation='softmax')
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 96, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Save the trained model
model.save("sound_classifier_CNN_Augmentedd.h5")
